In [4]:
from openai import OpenAI
import openai

# Replace with your actual API key
# api_key = ""
client = OpenAI()

query = f"""Problem Statement: Use LLMs for identifying the table and Key Value(KV) from provided invoices samples. We will be providing api keys for OpenAI to individual teams, with strong monitoring in the usage. If highly overused by any participants, we hold the right to disqualify them from the competition. So we request you to make the openai call carefully. 

KV: For KV, you are required to extract bbox and text for the following keys:
Invoice Number
Issue Date
Total
Table: For Table, you are required to identify the table bbox and table text.

Evaluation Criteria: 


The evaluation will be done via the mean of the kv and table extraction.
For KV, the extracted text is compared with ground truth text and fuzzy match should be > 80%. The bbox IOU should also be > 80%. This criteria should be crossed for each 3 KV field. Then, it is considered as a correct KV extraction i.e KV = 1.
For the table, the extracted table text will be compared with the ground truth text. The fuzzy match between these should be > 80%. And IOU of the table bbox > 80% with the ground truth to be considered as a valid extraction i.e, Table = 1.
For eg, for a single document:
If KV = 1 , Table = 1: the extraction score is 1 
If KV = 0, Table = 1, the extraction score is 0.5
In the similar fashion, the score is calculated for all the documents and then the final score is calculated by taking the mean of it.
"""

# Replace with the actual student's answer image URL
student_answer_image_url = "https://raw.githubusercontent.com/thenaivekid/autograder/main/sample%20student%20answer.jpg?token=GHSAT0AAAAAACJ6HTOOXTTITVSN46663QMWZLKVEEA"

# Create a request to the OpenAI API
response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": f"""**Role:** System

                    *Content:* 
                    {query}
                    """
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": """https://parade.com/.image/c_limit%2Ccs_srgb%2Cq_auto:good%2Cw_700/MTkwNTc1ODgzNTIyODExMDA1/all-that-you-will-be.webp"""
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

# Print the response
print(response.choices[0])


Choice(finish_reason=None, index=0, message=ChatCompletionMessage(content='I\'m sorry, but I cannot assist with the problem statement you provided, as it involves personal identification and the image shown does not relate to invoices or data extraction tasks. However, if you have an invoice or any other appropriate document image, I can help describe the image and provide general information about how you might go about identifying tables and key-value pairs. \n\nRegarding the image displayed, it shows two individuals in a tender embrace seated on a beanbag. There\'s a quote overlaid on the image saying "I love you for all that you are, and all that you have been and all that you will be." - Unknown. There\'s also a guitar leaning against the wall, suggesting a cozy and artistic setting. Please let me know if you have any questions or need further assistance with something else.', role='assistant', function_call=None, tool_calls=None), finish_details={'type': 'stop', 'stop': '<|fim_su

In [3]:
from openai import OpenAI
client = OpenAI()

query = "teach me to use pypdf in great details, i am using it to extract data from pdf to key value pair. also teach me to train layoutlm model"
response = client.chat.completions.create(
  model="gpt-4-0613",
  # response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": f"you are a helpful assistant"},
    {"role": "user", "content": f"{query}"}
  ]
)
print(response.choices[0].message.content)

Sure, I'd be happy to help guide you through this process. This tutorial will be quite long since the topics covered are extensive.

1. PyPDF2
PyPDF2 is a pure-python PDF library capable of splitting, merging, cropping, and transforming the pages of PDF files. It can also add custom data, viewing options, and passwords to PDF files.

Here is a simple example of using PyPDF2 to read a PDF:

```python
from PyPDF2 import PdfFileReader

def extract_information(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf = PdfFileReader(file)
        info = pdf.getDocumentInfo()
        number_of_pages = pdf.getNumPages()

    txt = f"""
    Information about {pdf_path}: 

    Author: {info.author}
    Creator: {info.creator}
    Producer: {info.producer}
    Subject: {info.subject}
    Title: {info.title}
    Number of pages: {number_of_pages}
    """

    print(txt)
    return info

if __name__ == '__main__':
    path = 'path_to_your_pdf.pdf'
    extract_information(path)
```
This will p

In [3]:
from PyPDF2 import PdfReader

def read_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    number_of_pages = len(reader.pages)
    text = ""
    for i in range(number_of_pages):
        page = reader.pages[i]
        text = page.extract_text()
    return text

pdf_path = '1040_file_1.pdf'
result_text = read_pdf(pdf_path)
print(result_text)

Form1040 2022 U.S. Individual Income Tax Return Department of the Treasury—Internal Revenue Service 
OMB No. 1545-0074 IRS Use Only—Do not write or staple in this space. 
Filing Status   
Check only 
one box.Single Married filing jointly Married filing separately (MFS) Head of household (HOH) Qualifying surviving 
spouse (QSS)
If you checked the MFS box, enter the name of your spouse. If you checked the HOH or QSS box, enter the child’s name if the qualifying 
person is a child but not your dependent: 
Your first name and middle initial Last name Your social security number 
If joint return, spouse’s first name and middle initial Last name Spouse’s social security number
Home address (number and street). If you have a P.O. box, see instructions. Apt. no. 
City, town, or post office. If you have a foreign address, also complete spaces below. State ZIP code
Foreign country name                                        Foreign province/state/county                        Foreign postal code

In [10]:
from PyPDF2 import PdfReader
import svgwrite

pdf_path = '1040_file_1.pdf'

reader = PdfReader(pdf_path)
page = reader.pages[0]

dwg = svgwrite.Drawing("GeoBase_test.svg", profile="tiny")


def visitor_svg_rect(op, args, cm, tm):
    if op == b"re":
        (x, y, w, h) = (args[i].as_numeric() for i in range(4))
        dwg.add(dwg.rect((x, y), (w, h), stroke="red", fill_opacity=0.05))


def visitor_svg_text(text, cm, tm, fontDict, fontSize):
    (x, y) = (tm[4], tm[5])
    dwg.add(dwg.text(text, insert=(x, y), fill="blue"))


page.extract_text(
    visitor_operand_before=visitor_svg_rect, visitor_text=visitor_svg_text
)
dwg.save()

In [8]:
from PyPDF2 import PdfReader

def extract_information(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf = PdfReader(file)
        info = pdf.
        number_of_pages = pdf.getNumPages()

    txt = f"""
    Information about {pdf_path}: 

    Author: {info.author}
    Creator: {info.creator}
    Producer: {info.producer}
    Subject: {info.subject}
    Title: {info.title}
    Number of pages: {number_of_pages}
    """

    print(txt)
    
pdf_path = '1040_file_1.pdf'

    
info = extract_information(pdf_path)

DeprecationError: getDocumentInfo is deprecated and was removed in PyPDF2 3.0.0. Use metadata instead.